Run: docker compose -f cass-docker-compose.yml up -d

In [13]:
import uuid
from cassandra.cluster import Cluster

In [47]:
# Otwieranie połączenia
cluster = Cluster(['cassandra'], port=9042)
session = cluster.connect()
print("Połączono z Cassandra")

Połączono z Cassandra


In [48]:
# Tworzenie Keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS lab_keyspace
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
""")
print("Utworzono keyspace")

Utworzono keyspace


In [49]:
# Użycie Keyspace
session.set_keyspace('lab_keyspace')

# Tworzenie tabeli
session.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id UUID PRIMARY KEY,
        name TEXT,
        email TEXT,
        age INT
    )
""")
print("Utworzono tabelę 'users'")

Utworzono tabelę 'users'


In [50]:
# Dodawanie danych
session.execute("""
    INSERT INTO users (user_id, name, email, age)
    VALUES (%s, %s, %s, %s)
""", (uuid.uuid4(), 'Anna Kowalska', 'anna@example.com', 28))

session.execute("""
    INSERT INTO users (user_id, name, email, age)
    VALUES (%s, %s, %s, %s)
""", (uuid.uuid4(), 'Jan Nowak', 'jan@example.com', 35))

# Can't extend by column hobby!
# session.execute("""
#     INSERT INTO users (user_id, name, email, age, hobby)
#     VALUES (%s, %s, %s, %s, %s)
# """, (uuid.uuid4(), 'Jan Nowak', 'jan@example.com', 35, 'tenis'))

print("Dodano dane")

Dodano dane


In [53]:
# Pobieranie danych
rows = session.execute("SELECT * FROM users")
print("\n Lista użytkowników:")
for row in rows:
    print(f"- {row.user_id} {row.name} ({row.age} lat) – {row.email}")



 Lista użytkowników:
- a130db24-551d-4caf-838e-5e4615c1a25c Anna Kowalska (36 lat) – anna@example.com
- 85d9319b-a95b-4de3-8257-15060b8a36cc Jan Nowak (35 lat) – jan@example.com


In [52]:
uid = "a130db24-551d-4caf-838e-5e4615c1a25c"
q = f"""
UPDATE users
SET age = 36
WHERE user_id = {uid};
"""
rows = session.execute(q)

In [40]:
# Usunięcie wszystkich wierszy
rows = session.execute("SELECT * FROM users")
for row in rows:
    q_del = f"""
    DELETE FROM users WHERE user_id = {row.user_id};
    """
    # print(q_del)
    rows = session.execute(q_del)

In [54]:
q_drop1 = """
DROP TABLE IF EXISTS users;
"""
rows = session.execute(q_drop1)

q_drop2 = """
DROP KEYSPACE IF EXISTS lab_keyspace;
"""
rows = session.execute(q_drop2)

In [55]:
# Zamknięcie połączenia
cluster.shutdown()